# Роли в промптах

## Обзор

В этом ноутбуке рассматривается концепция ролевого программирования в языковых моделях — как назначать модели конкретную роль и составлять эффективные описания ролей. Для демонстрации мы будем использовать GPT-модели OpenAI и библиотеку LangChain.

## Мотивация

Ролевое программирование — это мощная техника инженерии промптов, позволяющая направлять языковую модель на определённую «персону» или область экспертизы. Это может значительно повысить качество и релевантность её ответов для специализированных задач.

## Ключевые компоненты

1. **Назначение роли:** приёмы для задания роли языковой модели.
2. **Формулировка описания роли:** как составить качественное и подробное описание.
3. **Задание контекста:** как давать модели необходимую вводную для роли.
4. **Формулировка задачи:** как чётко объяснить, что требуется от модели в данной роли.

---

## Установка окружения

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

from dotenv import load_dotenv
load_dotenv(os.path.join(os.path.dirname(os.getcwd()), ".env"))
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('BASE_URL')

llm = ChatOpenAI(model="gpt-4o-mini", base_url=base_url)

## Базовое назначение роли

Давайте начнём с простого примера назначения роли. Мы создадим промпт, который назначает модели роль технического писателя.

In [ ]:
tech_writer_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""Вы — методист, специализирующийся на создании понятных и лаконичных учебных материалов для образовательных курсов в сфере ИТ.
Ваша задача — написать краткое объяснение понятия «{topic}» для обучающего пособия.
Сформулируйте объяснение в 2–3 простых предложениях, чтобы это было понятно новичкам без технической подготовки."""
)

chain = tech_writer_prompt | llm
response = chain.invoke({"topic": "облачные вычисления"})
print(response.content)

## Как создавать эффективные описания ролей

Давайте разберёмся, как формулировать более подробные и эффективные описания ролей. Мы создадим промпт для роли финансового консультанта с расширенным описанием.

In [ ]:
education_specialist_prompt = PromptTemplate(
    input_variables=["student_context"],
    template="""Вы — опытный методист с более чем 20-летним стажем в области образования и разработки учебных программ.
Вы помогаете ученикам с разным уровнем подготовки достигать учебных целей, делая сложные темы доступными.
Ваш подход включает в себя:
1. Внимательный анализ индивидуальных особенностей и уровня ученика
2. Ясное и простое объяснение даже сложных понятий
3. Заботу об этике и поддерживающей атмосфере
4. Акцент на долгосрочное понимание и самостоятельное мышление

Изучите следующую ситуацию ученика и дайте краткую (3-4 предложения) рекомендацию или совет:
{student_context}

Ваш ответ должен отражать ваш профессионализм и придерживаться описанного подхода."""
)

chain = education_specialist_prompt | llm
response = chain.invoke({"student_context": "Ученик 8 класса испытывает трудности с пониманием основ алгебры, несмотря на старание и выполнение домашних заданий."})
print(response.content)

## Сравнение ответов с разными ролями

Чтобы продемонстрировать, как разные роли влияют на ответы ИИ, давайте создадим промпты для трёх разных ролей и сравним их выводы на одну и ту же тему.

In [ ]:
roles = [
    ("Научный сотрудник", "Вы — исследователь в области педагогики и образовательных технологий. Объясните следующий термин с научной точки зрения:"),
    ("Учитель", "Вы — школьный учитель, который объясняет материал ученикам 7 класса. Объясните следующий термин простыми словами, чтобы было понятно подросткам:"),
    ("Журналист", "Вы — журналист, пишущий для популярного образовательного журнала. Объясните следующий термин понятно и интересно для широкой взрослой аудитории:")
]

topic = "Формирующее оценивание"

for role, description in roles:
    role_prompt = PromptTemplate(
        input_variables=["topic"],
        template=f"{description} {{topic}}"
    )
    chain = role_prompt | llm
    response = chain.invoke({"topic": topic})
    print(f"\nОбъяснение роли: {role}\n")
    print(response.content)
    print("-" * 50)

## Улучшаем описание ролей

Давайте рассмотрим, как уточнение ролей задаёт более специфичный результат. В качестве примера используем креативное письмо с разными стилями повествования.

In [ ]:
teacher_prompt = PromptTemplate(
    input_variables=["style", "scenario"],
    template="""Вы — выдающийся педагог, умеющий доносить материал в самых разных стилях преподавания.
Ваша задача — объяснить учебную ситуацию в стиле: {style}.
Ключевые особенности этого стиля:
1. {style_char1}
2. {style_char2}
3. {style_char3}

Напишите краткое объяснение (3–4 предложения) в этом стиле по следующей образовательной ситуации:
{scenario}

Ваш текст должен чётко отражать выбранный стиль объяснения."""
)

styles = [
    {
        "name": "Доброжелательный наставник",
        "char1": "Тёплая и поддерживающая манера",
        "char2": "Упор на веру в возможности ученика",
        "char3": "Побуждение не бояться ошибок"
    },
    {
        "name": "Строгий классический учитель",
        "char1": "Чёткая и структурированная подача",
        "char2": "Акцент на дисциплине и порядке",
        "char3": "Использование авторитетных формулировок"
    }
]

scenario = "Ученик не понимает новую тему по математике и стесняется задавать вопросы на уроке"

for style in styles:
    chain = teacher_prompt | llm
    response = chain.invoke({
        "style": style["name"],
        "style_char1": style["char1"],
        "style_char2": style["char2"],
        "style_char3": style["char3"],
        "scenario": scenario
    })
    print(f"\nВариант стиля: {style['name']}\n")
    print(response.content)
    print("-" * 50)